In [1]:
import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as display
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.utils import load_config

config_path = os.path.join('..\config\config.yaml')
config = load_config(config_path)

In [2]:
relative_path = config['yfinance']['csv_paths']['core_stock_data']
absolute_path = os.path.join(project_root, relative_path)

core_df = pd.read_csv(absolute_path)
core_df.head()

,Date,Close,Volume,SMA,EMA,RSI,Ticker
0,2019-03-14,45.932499,94318000,41.35925,42.219051,147.119051,AAPL
1,2019-03-15,46.529999,156171600,41.50025,42.388107,142.640938,AAPL
2,2019-03-18,47.005001,104879200,41.72940,42.569162,137.034379,AAPL
3,2019-03-19,46.632500,126585600,41.92075,42.728509,156.260813,AAPL
4,2019-03-20,47.040001,124140800,42.12190,42.897587,132.245226,AAPL


In [3]:
core_df['Date'] = pd.to_datetime(core_df['Date'])
core_df.set_index('Date', inplace = True)

In [4]:
fig = px.histogram(core_df, x = 'Close', title = 'Histogram of Close Prices for Core Stock Data')
fig.show()

In [5]:
fig = px.histogram(core_df, x = 'Close', facet_col= 'Ticker', nbins = 50, title = 'Histogram of Close Prices by Ticker for Core Stock Data')
fig.show()

In [6]:
fig = px.histogram(core_df, x = 'Close', color = 'Ticker', nbins = 50, barmode = 'overlay', title = 'Histogram of Close Prices by Ticker (Overlay) for Core Stock Data')
fig.show()

In [7]:
fig = go.Figure()

for ticker in core_df['Ticker'].unique():
    ticker_data = core_df[core_df['Ticker'] == ticker]
    fig.add_trace(go.Scatter(x = ticker_data.index, y = ticker_data['Close'], mode = 'lines', name = ticker))

fig.update_layout(title = 'Time Series Showing Daily Closing Prices by Ticker for Core Stock Data', xaxis_title = 'Date', yaxis_title = 'Closing Price')
fig.show()

In [8]:
volume_total = core_df.groupby('Ticker')['Volume'].sum().reset_index()

fig = px.bar(volume_total, x = 'Ticker', y = 'Volume', title = 'Bar Plot Showing Volume by Ticker for Core Stock Data')
fig.show()

In [9]:
fig = px.histogram(core_df, x = 'Volume', color = 'Ticker', nbins = 50, barmode = 'overlay', title = 'Histogram Showing Volume by Ticker (Overlay) for Core Stock Data')
fig.show()

In [12]:
#core_df.set_index('Date', inplace = True)
core_df_monthly = core_df.groupby('Ticker').resample('ME').sum().reset_index()
#core_df.set_index('Date', inplace = True)

fig = go.Figure()

for ticker in core_df_monthly['Ticker'].unique():
    ticker_data = core_df_monthly[core_df['Ticker'] == ticker]
    fig.add_trace(go.Scatter(x = ticker_data['Date'], y = ticker_data['Volume'], mode = 'lines', name = ticker))

fig.update_layout(title = 'Time Series Showing Monthly Volume by Ticker for Core Stock Data', xaxis_title = 'Date', yaxis_title = 'Volume')
fig.show()

ValueError: cannot insert Ticker, already exists

In [ ]:
fig = go.Figure()

for ticker in core_df['Ticker'].unique():
    ticker_data = core_df[core_df['Ticker'] == ticker]
    
    fig.add_trace(go.Scatter(x = ticker_data['Date'], y = ticker_data['Close'], mode = 'lines', name = f"{ticker} Close", line = dict(color = 'grey', width =1), opacity = 0.4))
    fig.add_trace(go.Scatter(x = ticker_data['Date'], y = ticker_data['SMA'], mode = 'lines', name = f"{ticker} SMA"))
    fig.add_trace(go.Scatter(x = ticker_data['Date'], y = ticker_data['EMA'], mode = 'lines', name = f"{ticker} EMA"))

fig.update_layout(title = "Moving Averages with Closing Prices Over Time for Core Stock Data",
                xaxis_title = "Date",
                yaxis_title = "Price",
                hovermode = "x")

fig.show()

In [ ]:
#core_df['Date'] = pd.to_datetime(core_df['Date'])
#core_df.set_index('Date', inplace = True)

rsi_monthly = core_df.groupby('Ticker').resample('ME').mean(numeric_only = True).reset_index()
rsi_monthly['Date'] = pd.to_datetime(rsi_monthly['Date'])
rsi_monthly.set_index('Date', inplace = True)

fig = go.Figure()

for ticker in core_df['Ticker'].unique():
    ticker_data = rsi_monthly[rsi_monthly['Ticker'] == ticker]
    
    fig.add_trace(go.Scatter(x = ticker_data.index, y = ticker_data['RSI'], mode = 'lines', name = f"{ticker} RSI"))
    
fig.update_layout(title = "Monthly RSI Over Time for Core Stock Data",
                xaxis_title = "Date",
                yaxis_title = "RSI",
                hovermode = "x")

fig.show()

In [ ]:
# While looking at the RSI for monthly certainly helps the traffic of the plot, we should still try to improve on this.  Let's remove those outliers and see if that helps.
lower_bound_rsi = rsi_monthly['RSI'].quantile(0.025)
upper_bound_rsi = rsi_monthly['RSI'].quantile(0.975)

rsi_monthly_filtered = rsi_monthly[(rsi_monthly['RSI'] >= lower_bound_rsi)  & (rsi_monthly['RSI'] <= upper_bound_rsi)].copy()

window_size = 14
rsi_monthly_filtered['Upper'] = rsi_monthly_filtered['RSI'].rolling(window = window_size).quantile(0.95)
rsi_monthly_filtered['Lower'] = rsi_monthly_filtered['RSI'].rolling(window = window_size).quantile(0.05)

fig = go.Figure()

for ticker in rsi_monthly_filtered['Ticker'].unique():
    ticker_data = rsi_monthly_filtered[rsi_monthly_filtered['Ticker'] == ticker]
    fig.add_trace(go.Scatter(x = ticker_data['Date'], y = ticker_data['RSI'], mode = "lines", name = f"{ticker} RSI"))

fig.update_layout(title="Monthly RSI Over Time for Core Stock Data (Revised)",
                xaxis_title="Date",
                yaxis_title="RSI",
                hovermode="x")

fig.show()

KeyError: 'Date'